<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

**Цель проекта:** защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 
Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df_duplicated = (df[df.duplicated()])
display(df_duplicated)
print('Доля дубликатов составляет', round(df_duplicated.shape[0] / df.shape[0] * 100, 2), '%')

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39.0,48100.0,1,0
488,1,24.0,32900.0,1,0
513,0,31.0,37400.0,2,0
718,1,22.0,32600.0,1,0
785,0,20.0,35800.0,0,0
...,...,...,...,...,...
4793,1,24.0,37800.0,0,0
4902,1,35.0,38700.0,1,0
4935,1,19.0,32700.0,0,0
4945,1,21.0,45800.0,0,0


Доля дубликатов составляет 3.06 %


Выделим целевой и незвисимые признаки:

In [6]:
RANDOM_STATE = 12345

In [7]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты'] #количество выплат за последние 5 лет

In [8]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, train_size=0.8, random_state = RANDOM_STATE)

display(target_train.count())
display(target_test.count())

4000

1000

Проведем маштабирование признаков 'Возраст', 'Зарплата', 'Члены семьи'

In [9]:
pd.options.mode.chained_assignment = None
scaler = StandardScaler()
numeric = ['Возраст', 'Зарплата', 'Члены семьи']
scaler.fit(features[numeric]) 
features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
display(features_train)
display(features_test)

,Пол,Возраст,Зарплата,Члены семьи
317,0,-1.297731,-0.011755,0.738401
4143,1,1.071948,-0.577463,-1.094314
4252,1,1.664368,1.099458,-0.177957
710,0,-0.349859,-0.082468,0.738401
148,0,-0.349859,-0.092570,-0.177957
...,...,...,...,...
3497,0,1.308916,-0.789604,-1.094314
3492,0,-0.349859,-1.739186,2.571115
2177,1,1.190432,0.483240,-0.177957
3557,0,-1.060763,1.028745,2.571115


,Пол,Возраст,Зарплата,Члены семьи
3183,0,0.242560,-0.092570,2.571115
1071,0,2.256787,0.321609,0.738401
2640,1,0.953464,0.220590,-1.094314
2282,0,-1.297731,-0.516851,-1.094314
1595,0,1.190432,0.008449,2.571115
...,...,...,...,...
3075,0,1.071948,0.301405,0.738401
1568,1,3.323143,-0.789604,-1.094314
2752,1,-0.468343,-1.900816,-0.177957
449,0,0.834980,-0.112774,-1.094314


Промежуточный вывод: 
- в датасете нет пропущенных значений в независимых и целевых признаках;
- значения признаков имеют числовой тип данных (целочисленный и с плавающей точкой);
- в датасете присутствуют дублирующиеся строки (3,06% от всех данных), что для цели текущего проекта несущественно, их оставим их без изменений;
- из датсета выделили целевой и независимые признаки.

## Умножение матриц

Умножим матрицу признаков на любую обратимую матрицу и проверим, измениться ли качество линейной регресии (задача обучения)

Обозначим:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** качество линейной регрессии не должно измениться

**Обоснование:**

Пусть
$$
w_1 = XP \;\;\; (1)
$$

тогда
$$
a_1 = XPw \;\;\; (2)
$$

Заменим в формуле обучения $XP$ на $L$:
$$
a_1 = XP((XP)^T XP)^{-1} (XP)^T y \;\;\; (3)
$$

Преобразуем выражение исходя из свойств матриц:

$$
a_1 = XP P^{-1}(P^T X^T X)^{-1} P^T X^T y \;\;\; (4)
$$

$$
P P^{-1} = E, \;\; XE = EX = X
$$

$$
a_1 = X(P^T X^T X)^{-1} P^T X^T y \;\;\; (5)
$$

$$
X^T X - квадратная \ матрица
$$

$$
P^T - квадратная \ матрица
$$

$$
a_1 = X (X^T X)^{-1} (P^T)^{-1} P^T X^T y \;\;\; (6)
$$


$$
(P^T)^{-1} P^T = E
$$

$$
a_1 = X (X^T X)^{-1} X^T y \;\;\; (7)
$$

$$
a = Xw \;\;\; или
$$

$$
a = X (X^T X)^{-1} X^T y \;\;\; (8)
$$

$$
Выражения \; (7)\; и \; (8)\; равны
$$

$$
a = a_1
$$

## Алгоритм преобразования

**Алгоритм**

1. В цикле создадим 40 матриц размера 4Х4 (обязательный параметр) со случайными элементами в диапазоне от 0 до 4.
2. Найдем обратные им матрицы и убедимся, что они являются единичными матрицами, в ином случае они будут вырожденными и в конечный список с подходящими матрицами не попадут.
3. Выберем случайную матрицу из полученных в результате выполнения цикла с условиями п.1 и п.2, что и будет конечным ключом шифрования признаков.
4. Сравним метрику R^2 на исходных и зашифрованных признаках.

**Обоснование:**

В датасете есть только 4 признака, поэтому для все генерируемые матрицы должны быть квадратными 4Х4. Случайный выбор элементов в матрицах и случайный выбор конечной матрицы для преобразования данных значительно снижают вероятность подбора метода дешифрования данных.

## Проверка алгоритма

Проверим качество модели с исходными данными:

In [10]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('Качество модели (R^2) с исходными данными:', round(r2_score(target_test, predictions), 2))

Качество модели (R^2) с исходными данными: 0.41


Проверим качество модели с преобразованными данными:

In [11]:
randomized = []
for i in range(40):
    try:
        i = np.random.randint(0, 4, size=(4, 4))
        k = np.linalg.inv(i)
        one = np.eye(4)
        m = i @ k
        if m[3].tolist() == one[3].tolist():
            if m[2].tolist() == one[2].tolist():
                if m[1].tolist() == one[1].tolist():
                    if m[0].tolist() == one[0].tolist():
                        randomized.append(i)
    except: 
        print('Найдена сингулярная матрица')

Найдена сингулярная матрица
Найдена сингулярная матрица
Найдена сингулярная матрица
Найдена сингулярная матрица
Найдена сингулярная матрица
Найдена сингулярная матрица


In [12]:
random_1 = randomized[random.randint(0, len(randomized))]
inverse = np.linalg.inv(random_1)
random_1 @ inverse

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [13]:
features_train_modify = features_train.values @ random_1
features_test_modify = features_test.values @ random_1

In [14]:
model1 = LinearRegression()
model1.fit(features_train_modify, target_train)
predictions = model1.predict(features_test_modify)
print('Качество модели (R^2) с преобразованными данными:', round(r2_score(target_test, predictions), 2))
print('Качество модели (R^2) с исходными данными:', round(r2_score(target_test, predictions), 2))

Качество модели (R^2) с преобразованными данными: 0.41
Качество модели (R^2) с исходными данными: 0.41


Вывод: качество модели не изменилось в результате шифрования признаков.

Проведем дешифрование данных для проверки:

In [15]:
original = features_test_modify @ np.linalg.inv(random_1)
a = pd.DataFrame(target_test)
b = pd.DataFrame(original, index = a.index, columns=features_test.columns).join(a)
ca = pd.DataFrame(predictions, index = a.index)
predict = []
for i in ca[0]:
    if i > 0.5:
        predict.append('1')
    else:
        predict.append('0')
b['Предсказания'] = predict
b.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты,Предсказания
3183,0.000000e+00,0.242560,-0.092570,2.571115,0,0
1071,0.000000e+00,2.256787,0.321609,0.738401,2,1
2640,1.000000e+00,0.953464,0.220590,-1.094314,0,0
2282,0.000000e+00,-1.297731,-0.516851,-1.094314,0,0
1595,0.000000e+00,1.190432,0.008449,2.571115,0,0
4456,1.000000e+00,0.361044,0.493342,-1.094314,0,0
3352,1.000000e+00,-0.942279,-0.779502,3.487473,0,0
3390,1.000000e+00,-0.112891,-1.506841,-0.177957,0,0
3431,1.000000e+00,1.901335,1.442924,1.654758,1,1
298,5.551115e-17,-1.297731,0.836808,-0.177957,0,0


## Вывод

В результате выполнения проекта было успешно проведено преобразование данных с целью защиты информации медотом генерации случайных матриц при неизменности качества модели как с исходными данными, так и с преобразованными.